# Baseline Generation Notebook

## Project: Finding and Solving Hard-to-Generate Examples - Speed Bumps

This notebook implements the baseline image generation pipeline using Stable Diffusion XL (SDXL)
to generate images of speed bumps and related roadway fixtures.

**Purpose:** Establish baseline performance before fine-tuning experiments.

**Framework:** PyTorch + HuggingFace Diffusers

**Model:** Stable Diffusion XL Base 1.0

**Author:** Based on HuggingFace Diffusers library (https://github.com/huggingface/diffusers)

**References:**
- Podell et al., "SDXL: Improving Latent Diffusion Models for High-Resolution Image Synthesis" (2023)
- HuggingFace Diffusers Documentation: https://huggingface.co/docs/diffusers


## Table of Contents
1. [Setup and Imports](#setup)
2. [Configuration](#config)
3. [Model Loading](#model)
4. [Prompt Collection](#prompts)
5. [Generation Pipeline](#generation)
6. [Batch Generation](#batch)
7. [Results and Evaluation](#results)


---
## 1. Setup and Imports {#setup}

Install required dependencies and import necessary libraries.

**Prerequisites:**
- Python 3.10+
- CUDA-capable GPU (recommended 10GB+ VRAM)
- Required packages listed in requirements.txt


In [1]:
# Install required dependencies (run once per environment)
# Note: You may need to restart the kernel after this cell finishes.

%pip install -q --upgrade pip
%pip install -q torch torchvision
%pip install -q "diffusers[torch]>=0.29.0" "transformers>=4.45.0" "accelerate>=0.34.0" safetensors pillow tqdm

# Quick import check
try:
    import diffusers, transformers, accelerate
    print("Dependencies installed and importable.")
except Exception as e:
    print("Dependency import check failed:", e)
    raise


Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Dependencies installed and importable.


In [2]:
# Standard library imports
import os
import json
from datetime import datetime
from pathlib import Path
from typing import List, Dict, Optional, Tuple
import warnings
warnings.filterwarnings('ignore')

# Numerical and image processing
import numpy as np
from PIL import Image

# Deep learning frameworks
import torch
from torch import autocast
from diffusers import StableDiffusionXLPipeline
from diffusers.utils import export_to_video

# Progress tracking
from tqdm import tqdm

print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"CUDA device: {torch.cuda.get_device_name(0)}")
    print(f"CUDA memory: {torch.cuda.get_device_properties(0).total_memory / 1024**3:.2f} GB")


'NoneType' object has no attribute 'cadam32bit_grad_fp32'
PyTorch version: 2.9.1
CUDA available: False


---
## 2. Configuration {#config}

Define configuration parameters for generation.

**Parameters:**
- `num_inference_steps`: Number of denoising steps (higher = better quality, slower)
- `guidance_scale`: How closely to follow the prompt (higher = more faithful to prompt)
- `width`, `height`: Output image dimensions (SDXL supports up to 1024x1024)
- `batch_size`: Number of images to generate per batch (adjust based on GPU memory)


In [3]:
# Generation parameters
CONFIG = {
    'model_id': 'stabilityai/stable-diffusion-xl-base-1.0',  # SDXL base model
    'num_inference_steps': 50,  # Number of denoising steps (default: 50)
    'guidance_scale': 7.5,  # Guidance scale (default: 7.5, range: 1-20)
    'width': 1024,  # Output image width (SDXL max: 1024)
    'height': 1024,  # Output image height (SDXL max: 1024)
    'batch_size': 1,  # Images per batch (adjust based on GPU memory)
    'device': 'cuda' if torch.cuda.is_available() else 'cpu',  # Use GPU if available
    'dtype': torch.float16 if torch.cuda.is_available() else torch.float32,  # Use fp16 on GPU for speed
}

# Output directories
OUTPUT_DIR = Path('results/baseline')
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)

LOGS_DIR = Path('logs')
LOGS_DIR.mkdir(parents=True, exist_ok=True)

# Reproducibility settings
SEED = 42  # Default seed for reproducibility
torch.manual_seed(SEED)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(SEED)

print(f"Configuration:")
print(f"  Model: {CONFIG['model_id']}")
print(f"  Device: {CONFIG['device']}")
print(f"  Output directory: {OUTPUT_DIR}")
print(f"  Seed: {SEED}")


Configuration:
  Model: stabilityai/stable-diffusion-xl-base-1.0
  Device: cpu
  Output directory: results/baseline
  Seed: 42


---
## 3. Model Loading {#model}

Load the SDXL pipeline. This downloads the model weights on first run (~6.9GB).

**Note:** Loading may take several minutes depending on internet speed.


In [4]:
# Load SDXL pipeline
print(f"Loading SDXL pipeline from {CONFIG['model_id']}...")
print("This may take several minutes on first run (model size ~6.9GB)...")

pipe = StableDiffusionXLPipeline.from_pretrained(
    CONFIG['model_id'],
    torch_dtype=CONFIG['dtype'],
    use_safetensors=True,
    variant="fp16" if CONFIG['dtype'] == torch.float16 else None,
)

# Move to device
pipe = pipe.to(CONFIG['device'])

# Optimize memory usage (optional but recommended)
pipe.enable_attention_slicing()  # Reduces VRAM usage at cost of slight speed
# pipe.enable_model_cpu_offload()  # Alternative: offload model to CPU when not in use

print("Model loaded successfully!")
print(f"Memory usage: {torch.cuda.memory_allocated() / 1024**3:.2f} GB" if torch.cuda.is_available() else "CPU mode")


Loading SDXL pipeline from stabilityai/stable-diffusion-xl-base-1.0...
This may take several minutes on first run (model size ~6.9GB)...


Loading pipeline components...:   0%|          | 0/7 [00:00<?, ?it/s]

Model loaded successfully!
CPU mode


---
## 4. Prompt Collection {#prompts}

Define positive and negative prompts for speed bump generation.

**Prompt Engineering Strategy:**
- Positive prompts: Descriptive phrases that specify speed bump characteristics
- Negative prompts: Terms to avoid unwanted artifacts (floating objects, distortions, etc.)


In [5]:
# Positive prompts - Various phrasings for speed bumps
POSITIVE_PROMPTS = [
    # Basic descriptions
    "A speed bump on a road",
    "A road with a speed bump",
    "Speed bump in the middle of a street",
    
    # Descriptive variations
    "A yellow-painted speed bump across a residential road",
    "A black asphalt speed bump integrated into a paved street",
    "A concrete speed hump on a neighborhood road",
    "A speed cushion on a narrow residential street",
    
    # Contextual variations
    "A speed bump in a parking lot with yellow markings",
    "A road hump with reflective strips in a suburban area",
    "A speed bump on a residential street during daytime",
    "A traffic calming speed bump on a city road",
    
    # Detailed descriptions
    "A raised speed bump made of rubber and asphalt, crossing a two-lane road",
    "A painted speed bump with white stripes, smoothly integrated into the road surface",
    "A speed bump with warning signs, on a paved street in good condition",
    "A concrete speed hump with rounded edges, designed for traffic calming",
    
    # Perspective variations
    "Front view of a speed bump on a road, photorealistic",
    "Aerial view of a speed bump on a residential street",
    "Side view of a speed bump integrated into asphalt road",
    "Close-up of a speed bump with road texture details",
    
    # Additional roadway fixtures
    "A speed table on a residential road",
    "Multiple speed bumps in a parking lot",
    "A raised crosswalk with speed bump characteristics",
    "A speed bump near a school zone with markings",
]

# Negative prompts - Terms to avoid unwanted artifacts
NEGATIVE_PROMPT = (
    "blurry, distorted, deformed, disfigured, poorly drawn, bad anatomy, "
    "wrong anatomy, extra limb, missing limb, floating limbs, disconnected limbs, "
    "mutation, mutated, ugly, disgusting, amputation, floating, disconnected, "
    "text, watermark, signature, out of focus, duplicate, morbid, mutilated, "
    "extra fingers, mutated hands, poorly drawn hands, poorly drawn face, "
    "artifacts, jpeg artifacts, compression artifacts, low quality, low resolution"
)

print(f"Total positive prompts: {len(POSITIVE_PROMPTS)}")
print(f"Sample prompts:")
for i, prompt in enumerate(POSITIVE_PROMPTS[:3], 1):
    print(f"  {i}. {prompt}")
print(f"\nNegative prompt: {NEGATIVE_PROMPT[:100]}...")


Total positive prompts: 23
Sample prompts:
  1. A speed bump on a road
  2. A road with a speed bump
  3. Speed bump in the middle of a street

Negative prompt: blurry, distorted, deformed, disfigured, poorly drawn, bad anatomy, wrong anatomy, extra limb, missi...


---
## 5. Generation Pipeline {#generation}

Define the core generation function.

**Function:** `generate_image(prompt, negative_prompt, seed, **kwargs)`
- Generates a single image from a text prompt
- Returns PIL Image and generation metadata
- Supports custom seeds for reproducibility


In [6]:
def generate_image(
    prompt: str,
    negative_prompt: str = NEGATIVE_PROMPT,
    seed: int = SEED,
    num_inference_steps: int = None,
    guidance_scale: float = None,
    width: int = None,
    height: int = None,
    save_path: Optional[Path] = None
) -> Tuple[Image.Image, Dict]:
    """
    Generate a single image from a text prompt using SDXL.
    
    Args:
        prompt: Positive text prompt describing the desired image
        negative_prompt: Negative prompt to avoid unwanted features
        seed: Random seed for reproducibility
        num_inference_steps: Number of denoising steps (uses CONFIG default if None)
        guidance_scale: Guidance scale (uses CONFIG default if None)
        width: Output image width (uses CONFIG default if None)
        height: Output image height (uses CONFIG default if None)
        save_path: Optional path to save the image
    
    Returns:
        Tuple of (PIL Image, metadata dictionary)
    """
    # Set random seed for reproducibility
    generator = torch.Generator(device=CONFIG['device']).manual_seed(seed)
    
    # Use config defaults if not specified
    num_inference_steps = num_inference_steps or CONFIG['num_inference_steps']
    guidance_scale = guidance_scale or CONFIG['guidance_scale']
    width = width or CONFIG['width']
    height = height or CONFIG['height']
    
    # Generate image with autocast for mixed precision
    with autocast(CONFIG['device']):
        result = pipe(
            prompt=prompt,
            negative_prompt=negative_prompt,
            num_inference_steps=num_inference_steps,
            guidance_scale=guidance_scale,
            width=width,
            height=height,
            generator=generator,
        )
    
    image = result.images[0]
    
    # Create metadata
    metadata = {
        'prompt': prompt,
        'negative_prompt': negative_prompt,
        'seed': seed,
        'num_inference_steps': num_inference_steps,
        'guidance_scale': guidance_scale,
        'width': width,
        'height': height,
        'model': CONFIG['model_id'],
        'timestamp': datetime.now().isoformat(),
    }
    
    # Save if path provided
    if save_path:
        image.save(save_path)
        # Save metadata as JSON
        metadata_path = save_path.with_suffix('.json')
        with open(metadata_path, 'w') as f:
            json.dump(metadata, f, indent=2)
    
    return image, metadata

print("Generation function defined successfully!")


Generation function defined successfully!


In [7]:
def generate_batch(
    prompts: List[str],
    negative_prompt: str = NEGATIVE_PROMPT,
    base_seed: int = SEED,
    use_varying_seeds: bool = True
) -> List[Dict]:
    """
    Generate images for a list of prompts.
    
    Args:
        prompts: List of positive prompts
        negative_prompt: Negative prompt to use for all generations
        base_seed: Base seed value (each prompt may use base_seed + index if varying)
        use_varying_seeds: If True, use different seed for each prompt
    
    Returns:
        List of metadata dictionaries for successful generations
    """
    generation_log = []
    timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
    
    print(f"Generating {len(prompts)} images...")
    print(f"Output directory: {OUTPUT_DIR}")
    
    for i, prompt in enumerate(tqdm(prompts, desc="Generating")):
        try:
            # Determine seed for this generation
            seed = base_seed + i if use_varying_seeds else base_seed
            
            # Create filename: timestamp_promptID_seed.png
            prompt_id = str(i).zfill(3)  # Zero-padded 3-digit ID
            filename = f"{timestamp}_{prompt_id}_{seed}.png"
            save_path = OUTPUT_DIR / filename
            
            # Generate image
            image, metadata = generate_image(
                prompt=prompt,
                negative_prompt=negative_prompt,
                seed=seed,
                save_path=save_path
            )
            
            # Add file info to metadata
            metadata['filename'] = filename
            metadata['prompt_id'] = prompt_id
            metadata['success'] = True
            
            generation_log.append(metadata)
            
        except Exception as e:
            print(f"\nError generating image for prompt {i}: {prompt}")
            print(f"Error: {str(e)}")
            
            # Log failure
            generation_log.append({
                'prompt_id': str(i).zfill(3),
                'prompt': prompt,
                'success': False,
                'error': str(e),
                'timestamp': datetime.now().isoformat(),
            })
            continue
    
    # Save generation log
    log_path = LOGS_DIR / f"generation_log_{timestamp}.json"
    with open(log_path, 'w') as f:
        json.dump({
            'timestamp': timestamp,
            'total_prompts': len(prompts),
            'successful': sum(1 for m in generation_log if m.get('success', False)),
            'failed': sum(1 for m in generation_log if not m.get('success', True)),
            'config': CONFIG,
            'generations': generation_log,
        }, f, indent=2)
    
    print(f"\nBatch generation complete!")
    print(f"Successful: {sum(1 for m in generation_log if m.get('success', False))}")
    print(f"Failed: {sum(1 for m in generation_log if not m.get('success', True))}")
    print(f"Generation log saved to: {log_path}")
    
    return generation_log

print("Batch generation function defined successfully!")


Batch generation function defined successfully!


---
## 7. Execute Batch Generation {#execute}

Run the batch generation process.

**Note:** This will generate images for all prompts in POSITIVE_PROMPTS.
Expected runtime: ~[X] seconds per image on [GPU model].


In [8]:
# Execute batch generation
# Uncomment the line below to run

# generation_log = generate_batch(
#     prompts=POSITIVE_PROMPTS,
#     negative_prompt=NEGATIVE_PROMPT,
#     base_seed=SEED,
#     use_varying_seeds=True
# )

print("Ready to generate. Uncomment the code above to execute batch generation.")


Ready to generate. Uncomment the code above to execute batch generation.


---
## 8. Test Single Generation {#test}

Optional: Test generation with a single prompt before running full batch.

Useful for:
- Verifying pipeline is working correctly
- Testing prompt variations
- Quick iteration on prompt engineering


In [9]:
# Test single generation
# Uncomment to test

# test_prompt = POSITIVE_PROMPTS[0]
# print(f"Testing with prompt: {test_prompt}")
# 
# test_image, test_metadata = generate_image(
#     prompt=test_prompt,
#     negative_prompt=NEGATIVE_PROMPT,
#     seed=42
# )
# 
# # Display image
# display(test_image)
# 
# print(f"\nMetadata:")
# print(json.dumps(test_metadata, indent=2))


---
## 9. Results Summary {#results}

View summary statistics from generation log.


In [10]:
# Load and display generation log summary
# Uncomment to view results

# import glob
# log_files = sorted(glob.glob(str(LOGS_DIR / 'generation_log_*.json')))
# 
# if log_files:
#     latest_log = log_files[-1]
#     with open(latest_log, 'r') as f:
#         log_data = json.load(f)
#     
#     print(f"Latest generation log: {latest_log}")
#     print(f"Total prompts: {log_data['total_prompts']}")
#     print(f"Successful: {log_data['successful']}")
#     print(f"Failed: {log_data['failed']}")
#     print(f"Success rate: {log_data['successful']/log_data['total_prompts']*100:.1f}%")
# else:
#     print("No generation logs found.")


---
## Notes and Troubleshooting

### Memory Issues
- If you encounter CUDA out of memory errors, try:
  - Reducing batch_size in CONFIG
  - Reducing image dimensions (width, height)
  - Enabling `enable_model_cpu_offload()` instead of `enable_attention_slicing()`

### Reproducibility
- Same seed + same prompt + same parameters = same output
- Use `use_varying_seeds=False` in `generate_batch()` to test reproducibility

### Output Organization
- Generated images: `results/baseline/`
- Metadata JSON files: `results/baseline/` (alongside images)
- Generation logs: `logs/generation_log_*.json`

### Performance
- Typical generation time: ~10-20 seconds per image on modern GPU
- CPU generation is much slower (~5-10 minutes per image)
- Consider batch size and generation count when estimating total time
